In [15]:
import pandas as pd

### Request relevant columns from dataset:

In [16]:
requested_columns = ["INCIDENT_DATE", "INCIDENT_NUMBER", "LOCATION_DISTRICT", "OFFENSE_DESCRIPTION", "WEAPON_TYPE", "ZIP", "LATITUDE", "LONGITUDE"]
lrpd = pd.read_csv("../data/lrpd.csv", usecols=requested_columns)
lrpd.set_index("INCIDENT_NUMBER", inplace=True)

### Drop duplicate rows:

In [17]:
prev_shape = lrpd.shape
lrpd = lrpd.drop_duplicates(keep='last')
new_shape = lrpd.shape

print(f'Dropped {(prev_shape[0] - new_shape[0])} duplicates!')

Dropped 5511 duplicates!


### Fill missing values with Number 0:

In [18]:
lrpd = lrpd.fillna(0)

### Parse Number 0 and String 1 to UNKNOWN weapon type:

In [19]:
lrpd["WEAPON_TYPE"] = lrpd["WEAPON_TYPE"].replace(0, "UNKNOWN")
lrpd["WEAPON_TYPE"] = lrpd["WEAPON_TYPE"].replace("1", "UNKNOWN")

### Convert columns to correct types:

In [20]:
lrpd["ZIP"] = pd.to_numeric(lrpd["ZIP"])
lrpd["LATITUDE"] = pd.to_numeric(lrpd["LATITUDE"])
lrpd["LONGITUDE"] = pd.to_numeric(lrpd["LONGITUDE"])
lrpd["INCIDENT_DATE"] = pd.to_datetime(lrpd["INCIDENT_DATE"])
lrpd["LOCATION_DISTRICT"] = pd.to_numeric(lrpd["LOCATION_DISTRICT"])

lrpd.head()

,INCIDENT_DATE,LOCATION_DISTRICT,OFFENSE_DESCRIPTION,WEAPON_TYPE,ZIP,LATITUDE,LONGITUDE
INCIDENT_NUMBER,,,,,,,
2021-092971,2021-08-04 10:21:00,72.0,THEFT FROM MOTOR VEHICLE,UNKNOWN,72204.0,0.000000,0.000000
2017-029450,2017-03-16 18:30:00,82.0,THEFT OF MOTOR VEHICLE PARTS,UNKNOWN,72206.0,34.649362,-92.301959
2017-156453,2017-09-01 12:00:00,53.0,RAPE,UNKNOWN,0.0,0.000000,0.000000
2019-130717,2019-10-17 21:00:00,51.0,RAPE,UNKNOWN,0.0,0.000000,0.000000
2020-073313,2020-07-02 19:10:00,50.0,RAPE,UNKNOWN,0.0,0.000000,0.000000


In [21]:
lrpd["WEEK_OF_MONTH"] = lrpd["INCIDENT_DATE"].dt.day // 7
lrpd["YEAR"] = lrpd["INCIDENT_DATE"].dt.year
lrpd["DAY"] = lrpd["INCIDENT_DATE"].dt.day
lrpd["DAY_OF_YEAR"] = lrpd["INCIDENT_DATE"].dt.dayofyear
lrpd.head()


,INCIDENT_DATE,LOCATION_DISTRICT,OFFENSE_DESCRIPTION,WEAPON_TYPE,ZIP,LATITUDE,LONGITUDE,WEEK_OF_MONTH,YEAR,DAY,DAY_OF_YEAR
INCIDENT_NUMBER,,,,,,,,,,,
2021-092971,2021-08-04 10:21:00,72.0,THEFT FROM MOTOR VEHICLE,UNKNOWN,72204.0,0.000000,0.000000,0,2021,4,216
2017-029450,2017-03-16 18:30:00,82.0,THEFT OF MOTOR VEHICLE PARTS,UNKNOWN,72206.0,34.649362,-92.301959,2,2017,16,75
2017-156453,2017-09-01 12:00:00,53.0,RAPE,UNKNOWN,0.0,0.000000,0.000000,0,2017,1,244
2019-130717,2019-10-17 21:00:00,51.0,RAPE,UNKNOWN,0.0,0.000000,0.000000,2,2019,17,290
2020-073313,2020-07-02 19:10:00,50.0,RAPE,UNKNOWN,0.0,0.000000,0.000000,0,2020,2,184


### Crime Risk Analysis

In [22]:
all_crimes = lrpd["OFFENSE_DESCRIPTION"].unique()
all_weapons = lrpd["WEAPON_TYPE"].unique()

In [23]:
violent_crimes = ['RAPE', 'AGGRAVATED ASSAULT', 'ALL OTHER LARCENY', 'ROBBERY', 'BURGLARY/B&E',
                  'MURDER & NONNEGLIGENT MANSLAUGHTER']
nonviolent_crimes = ['THEFT FROM MOTOR VEHICLE', 'MOTOR VEHICLE THEFT', 'THEFT OF MOTOR VEHICLE PARTS',
                     'SHOPLIFTING', 'THEFT FROM BUILDING', 'POCKET-PICKING', 'THEFT FROM COIN-OPERATED MACHINE',
                     'PURSE-SNATCHING']

In [24]:
def determine_crime_type(crime):
    if crime in violent_crimes:
        return 'Violent Crime'
    elif crime in nonviolent_crimes:
        return 'Non-Violent Crime'
    else:
        return 'Crime Type Unknown'

In [25]:
def determine_risk_type(crime):
    print(crime)

In [26]:
lrpd["CRIME_TYPE"] = lrpd["OFFENSE_DESCRIPTION"].apply(determine_crime_type)

In [27]:
lrpd.loc[(lrpd['CRIME_TYPE'] == 'Violent Crime') | (
        lrpd['WEAPON_TYPE'] != 'UNKNOWN'), "RISK_TYPE"] = "High Risk"
lrpd.loc[(lrpd['CRIME_TYPE'] == 'Violent Crime') & (
        lrpd['WEAPON_TYPE'] != 'UNKNOWN'), "RISK_TYPE"] = "High Risk"
lrpd.loc[(lrpd['CRIME_TYPE'] == 'Non-Violent Crime') & (
        lrpd['WEAPON_TYPE'] != 'UNKNOWN'), "RISK_TYPE"] = "High Risk"
lrpd.loc[(lrpd['CRIME_TYPE'] == 'Non-Violent Crime') & (
        lrpd['WEAPON_TYPE'] == 'UNKNOWN'), "RISK_TYPE"] = "Low Risk"

### Binary Classification

In [28]:
lrpd['RISK_TYPE_BC'] = lrpd["RISK_TYPE"].map({'High Risk': 1, 'Low Risk': 0})

### Saving Changes

In [29]:
# Save the cleaned dataset.
lrpd.to_csv("../data/lrpd-clean.csv")